# Working with AutoGen Agents

This lesson explores how to work with AutoGen's agent system, focusing on creating and using different types of agents for various tasks.

## Setup

In [1]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [2]:
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("Async environment configured for Jupyter.")
except ImportError:
    print("Please install nest_asyncio with `pip install nest_asyncio`")

Async environment configured for Jupyter.


In [3]:
import os
import getpass
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

## Creating a Basic Assistant Agent

Let's create a simple assistant agent that can analyze data and provide insights:

In [4]:
# Create a custom tool for data analysis
async def analyze_numbers(numbers: str) -> str:
    """Analyze a list of numbers and return basic statistics"""
    nums = [float(n) for n in numbers.split(',')]
    mean = sum(nums) / len(nums)
    maximum = max(nums)
    minimum = min(nums)
    return f"Analysis Results:\nMean: {mean:.2f}\nMax: {maximum}\nMin: {minimum}"

# Initialize the OpenAI client
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
)

# Create the assistant agent
data_analyst = AssistantAgent(
    name="data_analyst",
    model_client=model_client,
    tools=[analyze_numbers],
    system_message="You are a data analysis assistant. Use the analyze_numbers tool to help users understand their numerical data."
)

## Using the Assistant Agent

Let's demonstrate how to interact with the agent:

In [7]:
async def run_analysis():
    # Create a message asking for analysis
    message = TextMessage(
        content="Can you analyze these numbers: 10,15,20,25,30",
        source="user"
    )
    
    # Get the response using streaming
    return await Console(
        data_analyst.on_messages_stream(
            [message],
            cancellation_token=CancellationToken()
        )
    )

# Run the analysis
response = await run_analysis()

In [8]:
response

Response(chat_message=ToolCallSummaryMessage(source='data_analyst', models_usage=None, content='Analysis Results:\nMean: 20.00\nMax: 30.0\nMin: 10.0', type='ToolCallSummaryMessage'), inner_messages=[ToolCallRequestEvent(source='data_analyst', models_usage=RequestUsage(prompt_tokens=90, completion_tokens=24), content=[FunctionCall(id='call_bia6ANY13YIlzbGItYSZHHCY', arguments='{"numbers":"10,15,20,25,30"}', name='analyze_numbers')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='data_analyst', models_usage=None, content=[FunctionExecutionResult(content='Analysis Results:\nMean: 20.00\nMax: 30.0\nMin: 10.0', call_id='call_bia6ANY13YIlzbGItYSZHHCY')], type='ToolCallExecutionEvent')])

In [9]:
# Below is an example response object showing the process until the final output
# Response(chat_message=ToolCallSummaryMessage(source='data_analyst', models_usage=None, content='Analysis Results:\nMean: 20.00\nMax: 30.0\nMin: 10.0', type='ToolCallSummaryMessage'), inner_messages=[ToolCallRequestEvent(source='data_analyst', models_usage=RequestUsage(prompt_tokens=90, completion_tokens=24), content=[FunctionCall(id='call_bia6ANY13YIlzbGItYSZHHCY', arguments='{"numbers":"10,15,20,25,30"}', name='analyze_numbers')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='data_analyst', models_usage=None, content=[FunctionExecutionResult(content='Analysis Results:\nMean: 20.00\nMax: 30.0\nMin: 10.0', call_id='call_bia6ANY13YIlzbGItYSZHHCY')], type='ToolCallExecutionEvent')])

## Working with Tools and External Data

Let's create an agent that can work with pandas dataframes:

In [14]:
import pandas as pd
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_experimental.tools.python.tool import PythonAstREPLTool

# Create a sample dataset
data = {
    'name': ['Alice', 'Bob', 'Charlie', 'David'],
    'age': [25, 30, 35, 40],
    'salary': [50000, 60000, 75000, 80000]
}
df = pd.DataFrame(data)

# Create a tool for pandas operations
pandas_tool = LangChainToolAdapter(
    PythonAstREPLTool(locals={"df": df})
)

# Create an agent that can analyze the dataframe
data_agent = AssistantAgent(
    name="data_analyst",
    tools=[pandas_tool],
    model_client=model_client,
    system_message="You are a data analyst. Use pandas to analyze the dataframe 'df'."
)

async def analyze_dataframe():
    question = "What is the average age in the dataset?"
    
    return await Console(
        data_agent.on_messages_stream(
            [TextMessage(content=question, source="user")],
            cancellation_token=CancellationToken()
        )
    )

response = await analyze_dataframe()
response
## Conclusion


Response(chat_message=ToolCallSummaryMessage(source='data_analyst', models_usage=None, content='32.5', type='ToolCallSummaryMessage'), inner_messages=[ToolCallRequestEvent(source='data_analyst', models_usage=RequestUsage(prompt_tokens=116, completion_tokens=22), content=[FunctionCall(id='call_2RbAUlmtGZxSyrH4gexdZUjv', arguments='{"query":"df[\'age\'].mean()"}', name='python_repl_ast')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='data_analyst', models_usage=None, content=[FunctionExecutionResult(content='32.5', call_id='call_2RbAUlmtGZxSyrH4gexdZUjv')], type='ToolCallExecutionEvent')])

In [15]:
response.chat_message.content

'32.5'

In [16]:
import numpy as np
np.mean([25, 30, 35, 40])

32.5

This lesson demonstrates the key concepts of working with AutoGen agents, including:
- Basic agent creation and interaction
- Using tools and external functions
- Working with data analysis capabilities

The examples show how to create specialized agents for different purposes while maintaining a clean and efficient implementation.